In [ ]:
!pip install librosa scikit-learn joblib matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Include a link to this Zenodo page - https://zenodo.org/record/1188976

In [ ]:
# Replace with your actual path after uploading
dataset_path = "/content/drive/MyDrive/RAVDESS/Audio_Speech_Actors_01-24"

In [ ]:
!pip install resampy

In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import models, layers


In [ ]:
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

def extract_features(file):
    audio, sr = librosa.load(file)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

X, y = [], []

for actor_folder in os.listdir(dataset_path):
    actor_path = os.path.join(dataset_path, actor_folder)
    for file in os.listdir(actor_path):
        if file.endswith(".wav"):
            emotion_code = file.split("-")[2]
            emotion = emotions.get(emotion_code)
            file_path = os.path.join(actor_path, file)
            features = extract_features(file_path)
            X.append(features)
            y.append(emotion)

print("Total samples:", len(X))

Total samples: 1440


In [ ]:
X = np.array(X)
y = LabelEncoder().fit_transform(y)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,592 (1.17 MB)

 Trainable params: 307,592 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.1350 - loss: 49.3116 - val_accuracy: 0.0729 - val_loss: 2.1669
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.1188 - loss: 11.1224 - val_accuracy: 0.1215 - val_loss: 2.0821
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1253 - loss: 5.6135 - val_accuracy: 0.1181 - val_loss: 2.0762
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1201 - loss: 3.9768 - val_accuracy: 0.1181 - val_loss: 2.0765
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1379 - loss: 3.1503 - val_accuracy: 0.1181 - val_loss: 2.0791
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.1094 - loss: 2.7899 - val_accuracy: 0.1181 - val_loss: 2.0773
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1272 - loss: 2.5811 - val_accuracy: 0.1181 - val_loss: 2.0775
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.1251 - loss: 2.4307 - val_accuracy: 0.1181 -

In [ ]:
model.evaluate(X_test, y_test)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1086 - loss: 2.0631 


[2.0671615600585938, 0.1180555522441864]

In [ ]:
model.save("simplecnn_emotion_model.h5")

we can also use random classifier but for now i am going with facebook pretrained model from hugging face

In [ ]:
!pip install transformers datasets librosa soundfile torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import librosa
import numpy as np
from tqdm import tqdm
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

def extract_input_values(path):
    speech, _ = librosa.load(path, sr=16000)
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    return inputs.input_values.squeeze(0)  # shape: (seq_len,)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
emotions = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}
label2id = {emo: i for i, emo in enumerate(emotions.values())}

X, y = [], []
dataset_path = "/content/drive/MyDrive/RAVDESS/Audio_Speech_Actors_01-24"

for actor_folder in tqdm(os.listdir(dataset_path)):
    actor_path = os.path.join(dataset_path, actor_folder)
    for file in os.listdir(actor_path):
        if file.endswith(".wav"):
            emotion_code = file.split("-")[2]
            emotion = emotions.get(emotion_code)
            if emotion:
                try:
                    features = extract_input_values(os.path.join(actor_path, file))
                    X.append(features)
                    y.append(label2id[emotion])
                except Exception as e:
                    print(f"Error with {file}: {e}")


100%|██████████| 24/24 [00:16<00:00,  1.46it/s]


In [ ]:
from torch.nn.utils.rnn import pad_sequence

X_padded = pad_sequence(X, batch_first=True)  # shape: (N, max_seq_len)
y_tensor = torch.tensor(y)


In [ ]:
from transformers import Wav2Vec2Model
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

base_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
base_model.eval()  # freeze if needed

class EmotionClassifier(nn.Module):
    def __init__(self, base_model, num_labels):
        super().__init__()
        self.base = base_model
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)

    def forward(self, x):
        with torch.no_grad():
            outputs = self.base(x).last_hidden_state
        pooled = outputs.mean(dim=1)
        return self.classifier(pooled)


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

model = EmotionClassifier(base_model, num_labels=len(emotions))
optimizer = Adam(model.parameters(), lr=1e-4)
loss_fn = CrossEntropyLoss()

dataset = TensorDataset(X_padded, y_tensor)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

model.train()
for epoch in range(3):
    total_loss = 0
    for batch in dataloader:
        inputs, labels = batch
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss / len(dataloader)}")
    torch.save(model.state_dict(), f"{epoch}_emotion_model.pt")

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Epoch 1: Loss = 2.044619483749072
Epoch 2: Loss = 1.993574822942416
Epoch 3: Loss = 1.952870093451606


In [ ]:
torch.save(model.state_dict(), "emotion_model.pt")
processor.save_pretrained("wav2vec2-processor")

[]

In [ ]:
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        inputs, labels = batch
        logits = model(inputs)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=list(emotions.values())))


              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00        96
        calm       0.25      0.97      0.39       192
       happy       0.21      0.56      0.30       192
         sad       0.43      0.05      0.08       192
       angry       0.67      0.11      0.20       192
     fearful       0.38      0.06      0.11       192
     disgust       0.42      0.18      0.25       192
   surprised       1.00      0.01      0.02       192

    accuracy                           0.26      1440
   macro avg       0.42      0.24      0.17      1440
weighted avg       0.45      0.26      0.18      1440



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!zip -r emotion_model.zip wav2vec2-processor/

  adding: wav2vec2-processor/ (stored 0%)
  adding: wav2vec2-processor/special_tokens_map.json (deflated 45%)
  adding: wav2vec2-processor/preprocessor_config.json (deflated 38%)
  adding: wav2vec2-processor/tokenizer_config.json (deflated 71%)
  adding: wav2vec2-processor/vocab.json (deflated 55%)


In [ ]:
from google.colab import files
files.download('emotion_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>